In [1]:
from IPython.display import display, HTML
from IPython.display import display, Markdown, clear_output
import pandas as pd
import datetime
import gmaps
import geopy.geocoders as geo
import ipywidgets as widgets
from arctic import Arctic

/Users/tomneugebauer/opt/anaconda3/envs/NLPenv/lib/python3.7/site-packages/arctic/store/_pandas_ndarray_store.py:8: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%run config.ipynb
# connect with all APIs
gmaps.configure(api_key=gmap_key)
store = Arctic(mongo_host=mongo_setting,connectTimeoutMS=9000)

In [3]:
galleries = store['Art'].read("galleries").data
events = store['Art'].read("events").data

In [5]:
# setup up the starting map
# geolocator = geo.AlgoliaPlaces(app_id=algo_key)
# location = geolocator.geocode("Darlinghurst, 2010")
# start_map = (location.latitude, location.longitude)
start_map = (-33.8732, 151.225)
# date today
today = datetime.datetime.today().date()

In [6]:
# events = pd.read_excel("events.xlsx",index_col=0)

In [7]:
# events.columns

In [8]:
# events.to_excel("events.xlsx")

In [9]:
# galleries = pd.read_excel("galleries.xlsx", index_col=0)

In [10]:
events2 = events.copy()


In [11]:
# events

In [12]:
# galleries

In [13]:
for event in events.index:
    try:
        gallery_id = event.split("|")[1]
        events2.loc[event,"Gallery"] = galleries.loc[gallery_id,"Name"]
        events2.loc[event,"Area"] = galleries.loc[gallery_id,"Area"]
        events2.loc[event,"SubArea"] = galleries.loc[gallery_id,"SubArea"]
        events2.loc[event,"Address"] = galleries.loc[gallery_id,"Address"]
        events2.loc[event,"Website"] = galleries.loc[gallery_id,"Website"]
    except:
#         print("Couldn't find "+gallery_id)
        pass

In [14]:
events2['Date'] = events2['Date'].dt.date

In [15]:
from_date = today - datetime.timedelta(days=2)
to_date = today + datetime.timedelta(days=7)

In [16]:
events2 = events2[(events2["Date"] >= from_date) & (events2["Date"] <= to_date)]

In [17]:
order = ['Gallery','Type', 'Title', 'Artists',  'Date', 'Start', 'End', 'Address','SubArea', 'Area', 'Website']
events2 = events2[order]

In [18]:
galleries_map = []
for event in events2.index:
    try:
        gallery = event.split("|")[1]
        if gallery in galleries.Latitude.dropna().index:
            lat = galleries.loc[gallery,"Latitude"]
            long = galleries.loc[gallery,"Longitude"]
            galleries_map.append({
                    'name': galleries.loc[gallery,"Name"],
                    'address': galleries.loc[gallery,"Address"],
                    'location': (lat, long)})
    except:
#         print("Couldn't find "+gallery_id)
        pass

In [19]:
gallery_locations = [gallery['location'] for gallery in galleries_map]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
</dl>
"""
gallery_info = [info_box_template.format(**gallery) for gallery in galleries_map]

In [20]:
# galleries

In [21]:

out3 = widgets.Output()
with out3:
    fig = gmaps.figure(center=start_map, zoom_level=13.5)
    layer = gmaps.marker_layer(gallery_locations, info_box_content=gallery_info)
    fig.add_layer(layer)
    display(fig)


In [22]:
# gmaps.

In [23]:
# display(events2)

In [24]:
# events2

In [25]:
next_days = []
for day in sorted(list(set(events2.Date))):
#     print(day.strftime('%A'))
    next_days.append(day.strftime('%A'))

In [26]:
dates = sorted(list(set(events2.Date)))

In [27]:
# events2[events2.Date == dates[0]]

In [28]:
# next_days

In [29]:
sub_tab=[widgets.Output() for i in range(len(dates))]
tab = widgets.Tab(sub_tab)
for i in range (len(dates)):
    tab.set_title(i, next_days[i])

    with sub_tab[i]:
        clear_output()
        global events_set
        global fig
        events_set = events2[events2.Date == dates[i]].copy()
        galleries_map = []
        for event in events_set.index:
            try:
                gallery = event.split("|")[1]
                if gallery in galleries.Latitude.dropna().index:
                    lat = galleries.loc[gallery,"Latitude"]
                    long = galleries.loc[gallery,"Longitude"]
                    galleries_map.append({
                            'name': galleries.loc[gallery,"Name"],
                            'address': galleries.loc[gallery,"Address"],
                            'location': (lat, long)})
            except:
        #         print("Couldn't find "+gallery_id)
                pass
        gallery_locations = [gallery['location'] for gallery in galleries_map]
        #         fig = gmaps.figure(center=start_map, zoom_level=13.5)
        gallery_info = [info_box_template.format(**gallery) for gallery in galleries_map]
        fig = gmaps.figure(center=start_map, zoom_level=13.5)
        layer = gmaps.marker_layer(gallery_locations, info_box_content=gallery_info)
        fig.add_layer(layer)
        clear_output()
        display(fig)
        events_set.Date = pd.to_datetime(events_set.Date).dt.strftime('%d/%m/%Y')
        events_set.Start = pd.to_datetime(events_set.Start).dt.strftime('%H:%M')
        events_set.End = pd.to_datetime(events_set.End).dt.strftime('%H:%M')
        events_set = events_set.style.hide_index()
#         display(out3)
        display(events_set)
#         fig

 

### GalleryMe

In [30]:
display(tab)

In [31]:
# display(HTML(events2.to_html()))

In [32]:
def load_style(widget):
    with out3:
        i = tab.selected_index
        events_set = events2[events2.Date == dates[i]]
        galleries_map = []
        for event in events_set.index:
            try:
                gallery = event.split("|")[1]
                if gallery in galleries.Latitude.dropna().index:
                    lat = galleries.loc[gallery,"Latitude"]
                    long = galleries.loc[gallery,"Longitude"]
                    galleries_map.append({
                            'name': galleries.loc[gallery,"Name"],
                            'address': galleries.loc[gallery,"Address"],
                            'location': (lat, long)})
            except:
        #         print("Couldn't find "+gallery_id)
                pass
        gallery_locations = [gallery['location'] for gallery in galleries_map]
        #         fig = gmaps.figure(center=start_map, zoom_level=13.5)
        gallery_info = [info_box_template.format(**gallery) for gallery in galleries_map]
#         fig = gmaps.figure(center=start_map, zoom_level=13.5)
        layer = gmaps.marker_layer(gallery_locations, info_box_content=gallery_info)
        fig.add_layer(layer)
        clear_output()
        display(fig)

In [33]:
tab.observe(load_style)

In [34]:
# from IPython.display import clear_output, display
# import ipywidgets as widgets


# # set up a dictionary of Output widgets
# outputs = {i: widgets.Output() for i in range(0,3)}

# # add the Output widgets as tab childen
# tab = widgets.Tab()
# tab.children = list(outputs.values())
# for i, title in outputs.items():
#     tab.set_title(i, 'Tab '+str(i))

# def print_on_select(widget):
# #     get the correct Output widget based on the index of the chosen tab
#     tab_idx = widget['new']
#     output_widget = outputs[tab_idx]
#     with output_widget:
#         clear_output()
#         print('running long function'+str(tab_idx))




# tab.observe(print_on_select, names='selected_index')

# display(tab)